# Create some test data
As hard as it might be to believe, there are a limited number of angry ferret photos available.  I was able to find 8. 
So, we have to do a data scientist trick of augmenting the data we have to make it seem like it's more than just 8 samples.
We're going to use a python library called Augmentor.  We could also do this with Keras on the fly but doing it this way to simulate a batch of training data.


## Our Ground Truth Images

In [ ]:
import glob
from IPython.display import Image, display

def showImages(path):
    for imageName in glob.glob(path + '*.jpg'): 
        display(Image(filename=imageName))
        print(imageName)

In [ ]:
# Nice Images
showImages('./images/nice/')

In [ ]:
# Angry Images
showImages('./images/angry/')

## Lets Augment Our Data

In [ ]:
%%bash
rm -rf ./data
mkdir ./data
mkdir ./data/train ./data/valid
mkdir ./data/train/angry ./data/train/nice
mkdir ./data/valid/angry ./data/valid/nice

In [ ]:
#Data Augmentation with Augmentor
!pip install Augmentor --upgrade

In [ ]:
import Augmentor

#nice training data
p = Augmentor.Pipeline("./images/nice")
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.flip_top_bottom(probability=0.1)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)
p.sample(1000)

#angry training data
p = Augmentor.Pipeline("./images/angry")
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.flip_top_bottom(probability=0.1)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)
p.sample(1000)

!mv ./images/nice/output/* ./data/train/nice/
!mv ./images/angry/output/* ./data/train/angry/
!rm -rf ./images/nice/output/ ./images/angry/output/

#nice validaiton data - 200 images
p = Augmentor.Pipeline("./images/nice")
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.flip_top_bottom(probability=0.1)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)
p.sample(200)

#angry validation data - 200 images
p = Augmentor.Pipeline("./images/angry")
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.flip_top_bottom(probability=0.1)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)
p.sample(200)

!mv ./images/nice/output/* ./data/valid/nice/
!mv ./images/angry/output/* ./data/valid/angry/
!rm -rf ./images/nice/output/ ./images/angry/output/


## Copy our data to S3...

In [ ]:
!aws s3 cp --recursive --quiet ./data/ s3://<<your bucket>>/OwnModel/data